In [ ]:
import openai
import json
import time

In [ ]:
def authentication():
    data = json.load(open('.auth.json'))
    openai.organization = data['openai']['organization']
    openai.api_key = data['openai']['api_key']
    return

In [ ]:


def send_message_openai(text, MODEL="gpt-3.5-turbo", temperature=0,
                               role="user", timeout=10):
    messages = [
        {"role": role, "content": text},
    ]
    try:
        response = openai.ChatCompletion.create(
                model=MODEL,
                messages=messages,
                temperature=temperature,
            )
    except openai.error.Timeout as e:
        # Handle timeout error, e.g. retry or log
        print(f"OpenAI API request timed out: {e}")
        return
    except openai.error.APIError as e:
        # Handle API error, e.g. retry or log
        print(f"OpenAI API returned an API Error: {e}")
        return
    except openai.error.APIConnectionError as e:
        # Handle connection error, e.g. check network or log
        print(f"OpenAI API request failed to connect: {e}")
        return
    except openai.error.InvalidRequestError as e:
        # Handle invalid request error, e.g. validate parameters or log
        print(f"OpenAI API request was invalid: {e}")
        return 
    except openai.error.AuthenticationError as e:
        # Handle authentication error, e.g. check credentials or log
        print(f"OpenAI API request was not authorized: {e}")
        return 
    except openai.error.PermissionError as e:
        # Handle permission error, e.g. check scope or log
        print(f"OpenAI API request was not permitted: {e}")
        return 
    except openai.error.RateLimitError as e:
        # Handle rate limit error, e.g. wait or log
        print(f"OpenAI API request exceeded rate limit: {e}")
        time.sleep(50)
        return
    except openai.error.ServiceUnavailableError as e:
        # Handle rate limit error, e.g. wait or log
        print(f"OpenAI API ServiceUnavailableError: {e}")
        return 
    except TimeoutError as e:
        # Handle rate limit error, e.g. wait or log
        print(f"TimeoutError: {e}")
        return 
    return response["choices"][0]["message"]["content"]

In [ ]:
def check_and_create_columns(df, column_names):
    for column_name in column_names:
        if column_name not in df.columns:
            df[column_name] = None

In [ ]:
def loop(df, nombre_archivo):
    # Preparación
    check_and_create_columns(df, ['Prompt_gpt3', 'Prompt_gpt4',])
    for index, row in df.iterrows():
        print(f"index: {index}")
        df.loc[index, f'Prompt_gpt3'] = send_message_openai(df.loc[index, 'question'])
        df.loc[index, f'Prompt_gpt4'] = send_message_openai(df.loc[index, 'question'], MODEL='gpt-4')

    df.to_excel(f'{nombre_archivo}_respuesta.xlsx', index=False)
    return df

In [ ]:
import pandas as pd

file_name = 'stuvoc_spanish_questions'

df = pd.read_excel(f'{file_name}.xlsx')

authentication()


In [ ]:
results_df = loop(df, file_name)